In [6]:
# Imports
import base64
import io
import zlib
from datetime import datetime

In [9]:
def decode_grib_file(encoded_data, output_path):
    """
    Decodes a base64 encoded and zlib compressed GRIB file and saves it to a specified location.

    Args:
    - encoded_data (str): The base64 encoded string of the zlib compressed GRIB file content.
    - output_path (str): The path where the decoded GRIB file should be saved.

    Returns:
    - str: Path to the saved file.
    """

    # Split the received data into lines and extract only the message content lines, skipping the order numbers
    lines = encoded_data.strip().split("\n")
    encoded_data_without_nrs = ''.join([lines[i] for i in range(len(lines)) if i % 3 == 1])

    # Decode the base64 encoded data
    zipped_data = base64.b64decode(encoded_data_without_nrs)

    # Decompress the zlib data to obtain the original GRIB file
    grib_binary = zlib.decompress(zipped_data)

    # Save the decoded data to the specified path
    with open(output_path, 'wb') as file:
        file.write(grib_binary)

    return output_path

In [11]:
# Copy all messages received from the inreach
TEXT_RECIEVED = """
0
UjFKSlFnQUFXZ0VBQUJ3Q1lwNy9nQUptQUFBYUFRd1NBQUVBREFvQUFBQVZBQUFBQUFBZ0FQOEFBQU1BQWdCZHdJRVpRSUFBZlFDQTJzQWZRQjlBUUFBQUFB
0
1
QUFBQklPQUFORkdOZklCeFlBcE5acFFBQTNOemMzUjFKSlFnQUFXZ0VBQUJ3Q1lwNy9nQUptQUFBYUFRd1NBQUVBTUFvQUFBQVZBQUFBQUFBZ0FQOEFBQU1B
1
2
QWdCZHdJRVpRSUFBZlFDQTJzQWZRQjlBUUFBQUFBQUFBQklPQUFORkdMK3lCeGFKcUFSVmdBQTNOemMzUjFKSlFnQUFXZ0VBQUJ3Q1lwNy9nQ0ZwQUFvYUFR
2
3
d1NBQUVBREFvQUFBQVZBQUFCQUFBZ0FQOEFBQU1BQWdCZHdJRVpRSUFBZlFDQTJzQWZRQjlBUUFBQUFBQUFBQklPQUFEQ09ZTStCeHdZQTFOeXdBQTNOemMz
3
4
UjFKSlFnQUFXZ0VBQUJ3Q1lwNy9nQ0ZwQUFvYUFRd1NBQUVBTUFvQUFBQVZBQUFCQUFBZ0FQOEFBQU1BQWdCZHdJRVpRSUFBZlFDQTJzQWZRQjlBUUFBQUFB
4
5
QUFBQklPQUFEQ1FRYUdCMEJzQk4zckFBQTNOemMzUjFKSlFnQUFXZ0VBQUJ3Q1lwNy9nQ0pwQUFvYUFRd1NBQUVBREFvQUFBQVZBQUFCQUFBZ0FQOEFBQU1B
5
6
QWdCZHdJRVpRSUFBZlFDQTJzQWZRQjlBUUFBQUFBQUFBQklPQUFEQ0p6K2VCMlZLU0xBRXdBQTNOemMzUjFKSlFnQUFXQUVBQUJ3Q1lwNy9nQ0pwQUFvYUFR
6
7
d1NBQUVBTUFvQUFBQVZBQUFCQUFBZ0FQOEFBQU1BQWdCZHdJRVpRSUFBZlFDQTJzQWZRQjlBUUFBQUFBQUFBQkFFQUFCQnk1NUFCb21RSzAxQU56YzNOdz09
7
"""

# Define the path where the GRIB file should be saved
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
OUTPUT_PATH = f'./decoded_grib_{current_time}.grb'

# Use the function to decode and save the GRIB file
df = decode_grib_file(TEXT_RECIEVED, OUTPUT_PATH)

error: Error -3 while decompressing data: incorrect header check